In [ ]:
from langchain_groq import ChatGroq


llm = ChatGroq(
    temperature=0, 
    groq_api_key='GROQ_API_KEY', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("Bacis of ml")
print(response.content)

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.metacareers.com/jobs/503684125178318/")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.




Software Engineer, iOS | Meta Careers

























Skip to main contentJobsAreas of WorkLocations Career Programs  How We Work BlogJobsBackAreas of WorkBy TechnologyFacebookInstagramMessengerBy TechnologyMeta QuestWhatsAppWorkplaceBy FieldAdvertising TechnologyAR and VRArtificial IntelligenceBusiness Development and PartnershipsCommunications and Public PolicyCreativeData and AnalyticsDesign and User ExperienceEnterprise EngineeringGlobal OperationsBy FieldInfrastructureLegal, Finance, Facilities and AdministrationPeople and RecruitingProduct ManagementResearchSales and MarketingSecuritySoftware EngineeringTechnical Program ManagementLocationsBackCareer ProgramsStudents & GradsPathway ProgramsBackHow We WorkCultureDiversityVeteransLeadershipInnovationBenefitsBlogCreate Career ProfileYou can create a Career Profile to get job suggestions, prepare for the interview process, and more.Create Career ProfileAlready have a Career Profile?Log inStudents & GradsPathway ProgramsCul

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer, iOS',
  'experience': '2+ years of programming experience in a relevant language, 1+ years experience building complex applications targeting iOS in production using its native languages and frameworks',
  'skills': ['iOS SDK',
   'User interfaces',
   'Infrastructure',
   'Tools supporting applications on the iPhone or iPad',
   'Multithreading programming',
   'Mobile memory management',
   'Performance debugging and benchmarking'],
  'description': 'We are seeking full-time iOS Engineers to join our mobile teams. As a Mobile Software Engineer, you will specialize in building elegant products on world-class technologies that bring our social experiences to hundreds of millions of people, anytime and anywhere.'}]

In [11]:
import pandas as pd

df = pd.read_csv("dummy_people_portfolios.csv")
df

,Name,Tech Stack,Portfolio URL
0,Alice Johnson,"Python, Django, PostgreSQL",https://alicejohnson.dev
1,Bob Smith,"JavaScript, React, Node.js",https://bobsmith.io
2,Charlie Lee,"C++, OpenGL, Unreal Engine",https://charlielee.tech
3,David Kim,"Java, Spring, MySQL",https://davidkim.dev
4,Eva Brown,"Ruby, Rails, SQLite",https://evabrown.dev
5,Frank Miller,"PHP, Laravel, MariaDB",https://frankmiller.dev
6,Grace Taylor,"Go, Docker, Kubernetes",https://gracetaylor.io
7,Henry White,"Swift, iOS, Firebase",https://henrywhite.dev
8,Ivy Green,"TypeScript, Angular, MongoDB",https://ivygreen.dev
9,Jack Black,"C#, .NET, Azure",https://jackblack.dev


In [40]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Tech Stack"],
                       metadatas=({"links": row["Portfolio URL"]}),
                       ids=[str(uuid.uuid4())])

In [49]:
# Since job is a list with one dictionary, we need to access the first element
job_details = job[0]

# Now we can access the 'skills' key from the dictionary
skills = job_details['skills']

# skills is already a list, so we can use it directly in the query
links = collection.query(query_texts=skills, n_results=2).get('metadatas', [])

print(links)

[[{'links': 'https://henrywhite.dev'}, {'links': 'https://oliviacooper.dev'}], [{'links': 'https://charlielee.tech'}, {'links': 'https://davidkim.dev'}], [{'links': 'https://gracetaylor.io'}, {'links': 'https://jackblack.dev'}], [{'links': 'https://henrywhite.dev'}, {'links': 'https://oliviacooper.dev'}], [{'links': 'https://charlielee.tech'}, {'links': 'https://miamoore.dev'}], [{'links': 'https://henrywhite.dev'}, {'links': 'https://oliviacooper.dev'}], [{'links': 'https://charlielee.tech'}, {'links': 'https://miamoore.dev'}]]


In [52]:
job

[{'role': 'Software Engineer, iOS',
  'experience': '2+ years of programming experience in a relevant language, 1+ years experience building complex applications targeting iOS in production using its native languages and frameworks',
  'skills': ['iOS SDK',
   'User interfaces',
   'Infrastructure',
   'Tools supporting applications on the iPhone or iPad',
   'Multithreading programming',
   'Mobile memory management',
   'Performance debugging and benchmarking'],
  'description': 'We are seeking full-time iOS Engineers to join our mobile teams. As a Mobile Software Engineer, you will specialize in building elegant products on world-class technologies that bring our social experiences to hundreds of millions of people, anytime and anywhere.'}]

In [58]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Max, a business development executive at XYZ Company. XYZ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XYZ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase portfolio: {link_list}
        Remember you are Max, BDE at XYZ Company. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert iOS Development Services for Your Mobile Teams

Dear Hiring Manager,

I came across your job posting for a Software Engineer, iOS, and I'm excited to introduce you to meta, a leading AI & Software Consulting company that can help you build world-class iOS applications. With our expertise in mobile software development, we can support your mobile teams in creating elegant products that bring social experiences to hundreds of millions of people.

Our team of skilled iOS engineers has extensive experience in building complex applications targeting iOS in production using native languages and frameworks. We possess expertise in iOS SDK, user interfaces, infrastructure, tools supporting applications on the iPhone or iPad, multithreading programming, mobile memory management, and performance debugging and benchmarking.

At meta, we have a proven track record of delivering high-quality mobile solutions that meet the needs of our clients. Our portfolio includes:

* https://henr